In [2]:
"""
一般而言，HA蜡烛图有三个可以用于确认趋势和买入时机的主要信号：
（1）红色的蜡烛伴随无向下“影线”，表明强劲的上升趋势；
（2）体型小、被上下影线环绕的蜡烛预示着趋势的变化：喜欢冒险的交易员可能会在这里买进或卖出，而其他人则会等到确认信号出现后再做交易;
（3）绿色的蜡烛表示下跌趋势，在没有变色前最好减少仓位或空仓等待；绿色的蜡烛伴随无向上“影线”，表明强劲的下跌趋势;
     在趋势发生变化之前不要做多。     
从本质上来看，HA蜡烛图提供的是一个延迟信号，因为相对滞后性，所以产生的交易信号比较可靠，消除了市场噪音，
让我们更容易把握趋势以及捕捉买卖机会，但也是因为滞后，所以进场不好把握。很多技术分析指标都存在这样的特点。
而原始K线图则最能实时反映价格变动，基本不存在滞后性，因此在实际应用中一般建议将原始K线图图和HA蜡烛图结合分析。
HA蜡烛图的图表的优势是更能明确地反映一个趋势是否开始及结束。但值得注意的是，HA蜡烛图并不一定比普通K线图好，
"""

#先引入后面可能用到的包（package）
import pandas as pd  
import numpy as np
import tushare as ts 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import json 
# from pyecharts import Kline,Line, Bar, Scatter,Overlap
#引入pyecharts画图使用的是0.5.11版本，新版命令需要重写

# %matplotlib inline   
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

class Kline(object):
    def __init__(self,name,n=300,leixin='d',path='D:\\Quant_Code\\data\\ETF\\final_index_code.txt'):
        self.pro = ts.pro_api()        
        self.name = name
        self.n = n 
        self.type = leixin
        # self.index = {'上证综指': '000001.SH','深证成指': '399001.SZ',\
        #               '沪深300': '000300.SH','创业板指': '399006.SZ',\
        #               '上证50': '000016.SH','中证500': '000905.SH',\
        #               '中小板指': '399005.SZ','上证180': '000010.SH'}
        self.index = self.read_file(path)
        self.stocks = dict()

    def read_file(self,path):
        with open(path,'r',encoding='gbk') as f:
            file_str = f.read()
            dic = json.loads(file_str)
            
            return dic
    
    def get_code(self):
        """
        获取代码
        """
        df =  self.pro.stock_basic(exchange='')
        
        codes = df.ts_code.values
        names = df.name.values 
        stock = dict(zip(names,codes))
        #合并指数和个股成一个字典
        self.stocks = dict(stock,**self.index)
        return self.stocks[self.name]
    #默认设定时间周期为当前时间往前推300个交易日
    #日期可以根据需要自己改动
    def  get_data(self):
        """
        获取数据
        """
        leixin = self.type 
        t = datetime.now()
        t0 = t - timedelta(self.n)
        start =  t0.strftime("%Y%m%d")
        end = t.strftime("%Y%m%d")
        code = self.get_code()
        #如果代码在字典self.index里,则取得的是指数数据
        if code in  self.index.values():
            if leixin == 'w':
                df = self.pro.index_weekly(ts_code=code,start_date=start,\
                                        end_date=end)
            else:
                df = self.pro.index_daily(ts_code=code,start_date=start,\
                                        end_date=end)
        #否则取得是个股数据，使用前复权
        else:
            if leixin == 'w':
                df = self.pro.weekly(ts_code=code, start_date=start, end_date=end)
            else:                
                df = ts.pro_bar(ts_code=code,adj='qfq',start_date=start,end_date=end)
                        
        #将交易日期设置为索引值
        df.index = pd.to_datetime(df.trade_date)
        df.sort_index(inplace=True)
        return df
    def cal_hadata(self):
        df = self.get_data()
        df['Typical_Price'] = (df.high + df.low + df.close) / 3
        df['VP'] = df['Typical_Price'] * df['vol']
        df['Total_VP'] = df['VP'].cumsum()
        df['Total_V'] = df['vol'].cumsum()
        df['VWAP'] = df['Total_VP'] / df['Total_V']
        #计算HA版K线
        df['ha_close'] = (df.close + df.open + df.high + df.low) / 4.0
        ha_open=np.zeros(df.shape[0])
        ha_open[0] = df.open[0]
        for i in range(1,df.shape[0]):
            ha_open[i]  = (ha_open[i-1] + df['ha_close'][i-1]) / 2 
        df.insert(1,'ha_open',ha_open)
        df['ha_high'] = df[['high','ha_open','ha_close']].max(axis=1)
        df['ha_low'] = df[['low','ha_open','ha_close']].min(axis=1)
        df = df.iloc[1:]
        
        return df
    def kline_plot(self,ktype=0,ma_1=5,ma_2=20):
        """
        画K线图
        """
        leixin = self.type
        from pyecharts import Kline,Line, Bar, Scatter,Overlap
        df = self.cal_hadata()
        date  = df.index.strftime("%Y%m%d").tolist()
        if ktype == 0:
            k_value = df[['open','close','low','high']].values.round(2)

        else:
            k_value = df[['ha_open','ha_close','ha_low','ha_high']].values.round(2)
        if ktype == 0:
            kline = Kline(self.name + '行情走势')
        else:
            kline = Kline(self.name + 'HA行情走势')
        if leixin == 'w':
            kline.add('周K线图',date,k_value,is_datazoom_show=True,is_splitline_show=False,xaxis_rotate=60, yaxis_rotate=30)
        else:
            kline.add('日K线图',date,k_value,is_datazoom_show=True,is_splitline_show=False,xaxis_rotate=60, yaxis_rotate=30)
        #加入5、20日均线
        df['ma20'] = df.close.rolling(ma_2).mean()
        df['ma5'] = df.close.rolling(ma_1).mean()
        
        line = Line()
        v0 =  df['ma5'].round(2).tolist()
        v1 =  df['ma20'].round(2).tolist()
        v2 = df['VWAP'].round(2).tolist()
        line.add(str(ma_1)+'均线',date,v0,is_symbol_show=False,line_width=2,xaxis_rotate=60, yaxis_rotate=30)
        line.add(str(ma_2)+'均线',date,v1,is_symbol_show=False,line_width=2,xaxis_rotate=60, yaxis_rotate=30)
        line.add('VWAP',date,v2,is_symbol_show=False,line_width=2,xaxis_rotate=60, yaxis_rotate=30)
        
        #成交量
        bar = Bar()
        bar.add('成交量',date,df['vol'],tooltip_tragger='axis',is_legend_show=False,\
               is_yaxis_show=False,yaxis_max = 5*max(df['vol']),xaxis_rotate=60, yaxis_rotate=30)
        overlap = Overlap()
        overlap.add(kline)
        overlap.add(line)
        overlap.add(bar,yaxis_index=1,is_add_yaxis=True)
        return overlap.render(r"D:\\Quant_Code\\data\\HAKline\\" + self.name +".html") 
         
            
if __name__ == '__main__':
    kline = Kline("新能源车",n=990,leixin='w')
#     print(kline.cal_hadata())
    kline.kline_plot(ktype=1,ma_1=10,ma_2=30)
    

In [ ]:
from instrument_utility import *

i = InstrumentUtility()
# print(i.stock_price_eval('20211230','牧原股份'))
# print(i.get_filt_code('20211229'))
print(i.get_ETF_filt())


In [2]:
from daily2week_data import *

d = Daily2WeekData(['D:\Quant_Code\data\ETF\daily_etf.csv','D:\Quant_Code\data\ETF\daily_etf_21_22.csv']) 
    
# codes,df = d.get_data_accu()
codes,df =d.daily2week()
print(df.tail())

            close_max  close_first  close_min  close_last    vol_sum  \
date                                                                   
2021-11-21      1.004        1.002      0.991       0.991  409037.88   
2021-11-28      1.011        0.984      0.984       0.991  464754.29   
2021-12-05      1.010        0.980      0.975       1.001  400189.94   
2021-12-12      1.018        1.010      1.004       1.004  430729.26   
2021-12-19      1.008        0.987      0.987       1.008  151274.87   

            pct_chg_last       code  
date                                 
2021-11-21       -0.8008  159739.SZ  
2021-11-28       -0.1008  159739.SZ  
2021-12-05       -0.8911  159739.SZ  
2021-12-12       -1.2783  159739.SZ  
2021-12-19        0.5988  159739.SZ  
